<a href="https://colab.research.google.com/github/2spoorthy/End-to-End-Data-Science-Project/blob/main/EndTOEnd_DataScience_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pickle
from fastapi import FastAPI
import uvicorn

In [ ]:
# Step 1: Data Collection (Simulated dataset)
data = {
    "feature1": np.random.rand(1000),
    "feature2": np.random.rand(1000),
    "feature3": np.random.rand(1000),
    "feature4": np.random.rand(1000),
    "target": np.random.randint(0, 2, 1000)
}
df = pd.DataFrame(data)

In [ ]:
# Step 2: Data Preprocessing
X = df.drop(columns=["target"])
y = df["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Step 3: Model Training
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
# Step 4: Model Evaluation
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")

Model Accuracy: 0.4750


In [ ]:
# Step 5: Save Model
with open("model.pkl", "wb") as model_file:
    pickle.dump((model, scaler), model_file)

In [ ]:
# Step 6: API Development
app = FastAPI()

@app.post("/predict")
def predict(features: dict):
    try:
        input_data = np.array([features["feature1"], features["feature2"], features["feature3"], features["feature4"]]).reshape(1, -1)
        with open("model.pkl", "rb") as model_file:
            loaded_model, loaded_scaler = pickle.load(model_file)
        scaled_input = loaded_scaler.transform(input_data)
        prediction = loaded_model.predict(scaled_input)
        return {"prediction": int(prediction[0])}
    except Exception as e:
        return {"error": str(e)}

In [ ]:
# Step 7: Run the API
import asyncio
import uvicorn

if __name__ == "__main__":
    # Use nest_asyncio to run the API within the existing event loop
    import nest_asyncio
    nest_asyncio.apply()
    uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [2121]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [2121]
